In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)

naics_primes = pd.read_csv('NAICS Prime Transactions.csv', low_memory=False)
naics_subs = pd.read_csv('NAICS Sub Transactions.csv', low_memory=False)
psc_primes = pd.read_csv('PSC Prime Transactions.csv', low_memory=False)
psc_subs = pd.read_csv('PSC Sub Transactions.csv', low_memory=False)

primes = pd.concat([naics_primes, psc_primes]).drop_duplicates()
subs = pd.concat([naics_subs, psc_subs]).drop_duplicates()

### Top agencies from USASpending

In [2]:
import pandasql as ps

query="""
SELECT
    awarding_agency_name,
    SUM(total_dollars_obligated) AS total_obligation,
    COUNT(*) AS number_of_transactions
FROM primes
WHERE awarding_agency_name IS NOT NULL
GROUP BY awarding_agency_name
ORDER BY total_obligation DESC, number_of_transactions DESC
"""

top_agencies = ps.sqldf(query, locals())
top_agencies.to_excel('top_agencies.xlsx', index=False)
top_agencies

,awarding_agency_name,total_obligation,number_of_transactions
0,Department of Justice,2.231935e+09,1441
1,Department of Veterans Affairs,4.516496e+08,728
2,Department of Homeland Security,2.631673e+08,59
3,Department of Health and Human Services,1.508402e+08,76
4,Court Services and Offender Supervision Agency,5.530342e+07,13
5,Department of Defense,4.743460e+07,46
6,Federal Trade Commission,6.977478e+06,5
7,Department of Labor,2.742545e+06,6
8,Small Business Administration,1.818884e+06,4
9,U.S. International Development Finance Corpora...,1.757374e+06,3


### How much did your top agencies spend on your products/services over the past fiscal year?

In [3]:
list_top_agencies = top_agencies['awarding_agency_name'][:3].tolist()
list_top_agencies = ', '.join(f"'{agency}'" for agency in list_top_agencies)

# How much are the top 3 agencies spending on NAICS codes
query = f"""
SELECT
    awarding_agency_name,
    naics_code,
    SUM(total_dollars_obligated) AS total_obligation,
    COUNT(*) AS number_of_transactions
FROM primes
WHERE 
    awarding_agency_name IN ({list_top_agencies})
GROUP BY awarding_agency_name, naics_code
ORDER BY awarding_agency_name DESC, total_obligation DESC, number_of_transactions DESC
"""

top_agencies_naics_spending = ps.sqldf(query, locals())
top_agencies_naics_spending.to_excel('top_agencies_naics_spending.xlsx', index=False)
top_agencies_naics_spending

,awarding_agency_name,naics_code,total_obligation,number_of_transactions
0,Department of Veterans Affairs,624221,2.190399e+08,401
1,Department of Veterans Affairs,623220,1.507659e+08,276
2,Department of Veterans Affairs,624120,4.330168e+07,4
3,Department of Veterans Affairs,541611,2.663022e+07,9
4,Department of Veterans Affairs,623990,7.317921e+06,5
5,Department of Veterans Affairs,561990,2.161465e+06,1
6,Department of Veterans Affairs,713940,1.022960e+06,16
7,Department of Veterans Affairs,621330,6.574738e+05,3
8,Department of Veterans Affairs,624410,3.172500e+05,2
9,Department of Veterans Affairs,812990,1.879269e+05,5


In [4]:
primes['product_or_service_code'] 

0      G004
1      G004
2      Q523
3      Q526
4      Q526
       ... 
269    R401
270    R401
271    R401
272    R401
273    R401
Name: product_or_service_code, Length: 2448, dtype: object

In [5]:
# How much are the top 3 agencies spending on PSC
query = f"""
SELECT
    awarding_agency_name,
    product_or_service_code,
    SUM(total_dollars_obligated) AS total_obligation,
    COUNT(*) AS number_of_transactions
FROM primes
WHERE 
    awarding_agency_name IN ({list_top_agencies})
    AND product_or_service_code = 'R401'
GROUP BY awarding_agency_name, product_or_service_code
ORDER BY awarding_agency_name DESC, total_obligation DESC, number_of_transactions DESC
"""

top_agencies_psc_spending = ps.sqldf(query, locals())
top_agencies_psc_spending.to_excel('top_agencies_psc_spending.xlsx', index=False)
top_agencies_psc_spending

,awarding_agency_name,product_or_service_code,total_obligation,number_of_transactions
0,Department of Veterans Affairs,R401,7.452592e+07,46
1,Department of Justice,R401,2.852904e+06,14
2,Department of Homeland Security,R401,2.629464e+08,54


### What percentage of their awards went to small businesses

In [6]:
query=f"""
SELECT
    awarding_agency_name,
    contracting_officers_determination_of_business_size,
    SUM(total_dollars_obligated) AS total_obligation,
    COUNT(*) AS number_of_transactions
FROM primes
WHERE 
    awarding_agency_name IN ({list_top_agencies}) 
    AND action_date_fiscal_year = '2024'
GROUP BY awarding_agency_name, contracting_officers_determination_of_business_size 
ORDER BY total_obligation DESC
"""

top_agencies_sb_spending = ps.sqldf(query, locals())
top_agencies_sb_spending

,awarding_agency_name,contracting_officers_determination_of_business_size,total_obligation,number_of_transactions
0,Department of Justice,OTHER THAN SMALL BUSINESS,1.984264e+09,1235
1,Department of Veterans Affairs,OTHER THAN SMALL BUSINESS,3.504353e+08,534
2,Department of Homeland Security,SMALL BUSINESS,1.404119e+08,28
3,Department of Justice,SMALL BUSINESS,1.118838e+08,114
4,Department of Homeland Security,OTHER THAN SMALL BUSINESS,1.049969e+08,23
5,Department of Veterans Affairs,SMALL BUSINESS,5.187554e+07,96


In [7]:
query = """
SELECT 
    sb.awarding_agency_name,
    sb.contracting_officers_determination_of_business_size,
    sb.total_obligation AS size_obligation,
    ta.total_obligation AS agency_total_obligation,
    ROUND((sb.total_obligation * 100.0 / ta.total_obligation), 2) AS percentage_of_total
FROM top_agencies_sb_spending AS sb
JOIN top_agencies AS ta
ON sb.awarding_agency_name = ta.awarding_agency_name
ORDER BY sb.awarding_agency_name, sb.contracting_officers_determination_of_business_size DESC
"""

top_agencies_sb_percentage = ps.sqldf(query, locals())
top_agencies_sb_percentage.to_excel('top_agencies_small_business_spending.xlsx', index=False)
top_agencies_sb_percentage

,awarding_agency_name,contracting_officers_determination_of_business_size,size_obligation,agency_total_obligation,percentage_of_total
0,Department of Homeland Security,SMALL BUSINESS,1.404119e+08,2.631673e+08,53.35
1,Department of Homeland Security,OTHER THAN SMALL BUSINESS,1.049969e+08,2.631673e+08,39.90
2,Department of Justice,SMALL BUSINESS,1.118838e+08,2.231935e+09,5.01
3,Department of Justice,OTHER THAN SMALL BUSINESS,1.984264e+09,2.231935e+09,88.90
4,Department of Veterans Affairs,SMALL BUSINESS,5.187554e+07,4.516496e+08,11.49
5,Department of Veterans Affairs,OTHER THAN SMALL BUSINESS,3.504353e+08,4.516496e+08,77.59


### Do your top agencies utilize small business set-aside and do your certifications match?

In [8]:
query = f"""
SELECT
    awarding_agency_name,
    type_of_set_aside,
    SUM(total_dollars_obligated) AS total_obligation,
    ROUND((SUM(total_dollars_obligated) * 100.0 / 
           (SELECT SUM(total_dollars_obligated) 
            FROM primes 
            WHERE awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
            AND awarding_agency_name = primes.awarding_agency_name
           )), 2) AS percentage_of_total
FROM primes
WHERE 
    awarding_agency_name IN ({list_top_agencies})
GROUP BY awarding_agency_name, type_of_set_aside
ORDER BY total_obligation DESC
"""

top_agencies_set_aside = ps.sqldf(query, locals())
top_agencies_set_aside.to_excel('top_agencies_set_aside_spending.xlsx', index=False)
top_agencies_set_aside

,awarding_agency_name,type_of_set_aside,total_obligation,percentage_of_total
0,Department of Justice,None,2.231485e+09,78.73
1,Department of Homeland Security,None,2.613625e+08,9.22
2,Department of Veterans Affairs,None,2.472063e+08,8.72
3,Department of Veterans Affairs,NO SET ASIDE USED.,1.967944e+08,6.94
4,Department of Veterans Affairs,SERVICE DISABLED VETERAN OWNED SMALL BUSINESS ...,3.518564e+06,0.12
5,Department of Veterans Affairs,SMALL BUSINESS SET ASIDE - TOTAL,3.472891e+06,0.12
6,Department of Homeland Security,NO SET ASIDE USED.,1.779907e+06,0.06
7,Department of Veterans Affairs,SDVOSB SOLE SOURCE,6.574738e+05,0.02
8,Department of Justice,SMALL BUSINESS SET ASIDE - TOTAL,2.343884e+05,0.01
9,Department of Justice,NO SET ASIDE USED.,2.159013e+05,0.01


### Who are your top competitors that are winning those awards? How does your business company compare? 

In [9]:
query="""
SELECT
    recipient_name,
    recipient_uei,
    SUM(total_dollars_obligated) AS total_obligation,
    count(*) AS number_of_transactions,
    contracting_officers_determination_of_business_size,
    organizational_type
FROM primes
GROUP BY recipient_uei
ORDER BY total_obligation DESC, number_of_transactions DESC
"""

top_recipients = ps.sqldf(query, locals())
top_recipients.to_excel('top_recipients.xlsx', index=False)
top_recipients

,recipient_name,recipient_uei,total_obligation,number_of_transactions,contracting_officers_determination_of_business_size,organizational_type
0,GEO REENTRY INC,KDQ3R3N44ZJ1,4.061654e+08,166,OTHER THAN SMALL BUSINESS,CORPORATE NOT TAX EXEMPT
1,DISMAS CHARITIES INC,HUFKC6ZWCEN8,3.755548e+08,272,OTHER THAN SMALL BUSINESS,CORPORATE TAX EXEMPT
2,"CORECIVIC, INC.",HJGMJN1JKL46,1.320150e+08,36,OTHER THAN SMALL BUSINESS,CORPORATE NOT TAX EXEMPT
3,COASTAL CLINICAL & MANAGEMENT SERVICES INC,NMKBMMA32Q76,1.228260e+08,17,SMALL BUSINESS,CORPORATE NOT TAX EXEMPT
4,"VOLUNTEERS OF AMERICA TEXAS, INC",NE4QKLL1Q6R9,1.135371e+08,27,OTHER THAN SMALL BUSINESS,CORPORATE TAX EXEMPT
...,...,...,...,...,...,...
410,"MILITARY AND VETERANS AFFAIRS, NEW JERSEY DEPA...",UJB9LRHNFKW6,0.000000e+00,1,OTHER THAN SMALL BUSINESS,US GOVERNMENT ENTITY
411,WILLIAMS SCOTSMAN INC,URUVKGK3FTV1,0.000000e+00,1,SMALL BUSINESS,CORPORATE NOT TAX EXEMPT
412,WJM PROFESSIONAL SERVICES LLC,WNRJMTM3AA66,0.000000e+00,1,SMALL BUSINESS,OTHER
413,FAMILY SERVICES INCORPORATED,XKN4WFGRN8F4,0.000000e+00,1,OTHER THAN SMALL BUSINESS,CORPORATE TAX EXEMPT


### Are there sub-contracting opportunities available from these competitors? 

In [10]:
primes_awardees = top_recipients['recipient_uei'].tolist()
primes_awardees = ', '.join(f"'{agency}'" for agency in primes_awardees)

query = f"""
SELECT
    prime_awardee_name,
    prime_awardee_uei,
    subawardee_name,
    subawardee_uei,
    SUM(prime_award_total_outlayed_amount) AS total_obligation,
    COUNT(*) AS number_of_transactions
FROM subs
WHERE prime_awardee_uei IN ({primes_awardees})
GROUP BY subawardee_uei
ORDER BY total_obligation DESC, number_of_transactions DESC
"""

sub_opportunities = ps.sqldf(query, locals())
sub_opportunities

,prime_awardee_name,prime_awardee_uei,subawardee_name,subawardee_uei,total_obligation,number_of_transactions


### How often does the agency utilize Simplified Acquisition Procedures (SAP)?

In [11]:
query="""
SELECT
    awarding_agency_name,
    simplified_procedures_for_certain_commercial_items,
    SUM(total_dollars_obligated) AS total_obligation,
    COUNT(*) AS number_of_transactions,
    ROUND(SUM(total_dollars_obligated) * 100.0 / SUM(SUM(total_dollars_obligated)) OVER (PARTITION BY awarding_agency_name), 2) AS percentage_of_obligation
FROM primes
WHERE 
    awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
    AND simplified_procedures_for_certain_commercial_items != "None"
GROUP BY awarding_agency_name, simplified_procedures_for_certain_commercial_items
"""

top_agencies_sap = ps.sqldf(query, locals())
top_agencies_sap.to_excel('top_agencies_sap_spending.xlsx', index=False)
top_agencies_sap

,awarding_agency_name,simplified_procedures_for_certain_commercial_items,total_obligation,number_of_transactions,percentage_of_obligation
0,Department of Health and Human Services,NO,1.303623e+08,55,86.42
1,Department of Health and Human Services,YES,2.047797e+07,17,13.58
2,Department of Justice,NO,2.229571e+09,1428,99.89
3,Department of Justice,YES,2.364400e+06,13,0.11
4,Department of Veterans Affairs,NO,2.675842e+08,389,59.25
5,Department of Veterans Affairs,YES,1.840653e+08,338,40.75


### Top agencies' preferred buying method

In [12]:
query="""
SELECT
    awarding_agency_name,
    award_type,
    COUNT(*) AS number_of_transactions,
    SUM(total_dollars_obligated) AS total_obligation
FROM primes
WHERE awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
GROUP BY awarding_agency_name, award_type
ORDER BY awarding_agency_name, total_obligation DESC, number_of_transactions DESC
"""

top_agencies_award_type = ps.sqldf(query, locals())
top_agencies_award_type.to_excel('top_agencies_award_type.xlsx', index=False)
top_agencies_award_type

,awarding_agency_name,award_type,number_of_transactions,total_obligation
0,Department of Health and Human Services,DEFINITIVE CONTRACT,16,1.119872e+08
1,Department of Health and Human Services,PURCHASE ORDER,10,2.049300e+07
2,Department of Health and Human Services,DELIVERY ORDER,25,1.574160e+07
3,Department of Health and Human Services,BPA CALL,17,2.618420e+06
4,Department of Health and Human Services,None,8,0.000000e+00
5,Department of Justice,DELIVERY ORDER,1435,2.231719e+09
6,Department of Justice,PURCHASE ORDER,5,2.159013e+05
7,Department of Justice,None,1,0.000000e+00
8,Department of Veterans Affairs,DELIVERY ORDER,553,2.579272e+08
9,Department of Veterans Affairs,DEFINITIVE CONTRACT,45,1.086056e+08


In [13]:
query="""
SELECT
    awarding_agency_name,
    idv_type,
    COUNT(*) AS number_of_transactions,
    SUM(total_dollars_obligated) AS total_obligation
FROM primes
WHERE awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
GROUP BY awarding_agency_name, idv_type
ORDER BY awarding_agency_name, total_obligation DESC, number_of_transactions DESC
"""

top_agencies_idv_type = ps.sqldf(query, locals())
top_agencies_idv_type.to_excel('top_agencies_idv_type.xlsx', index=False)
top_agencies_idv_type

,awarding_agency_name,idv_type,number_of_transactions,total_obligation
0,Department of Health and Human Services,None,68,1.508402e+08
1,Department of Health and Human Services,BPA,6,0.000000e+00
2,Department of Health and Human Services,IDC,2,0.000000e+00
3,Department of Justice,None,1440,2.231935e+09
4,Department of Justice,IDC,1,0.000000e+00
5,Department of Veterans Affairs,None,722,4.516496e+08
6,Department of Veterans Affairs,IDC,5,0.000000e+00
7,Department of Veterans Affairs,BPA,1,0.000000e+00


In [14]:
query="""
SELECT
    awarding_agency_name,
    type_of_idc,
    COUNT(*) AS number_of_transactions,
    SUM(total_dollars_obligated) AS total_obligation
FROM primes
WHERE awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
GROUP BY awarding_agency_name, type_of_idc
ORDER BY awarding_agency_name, total_obligation DESC, number_of_transactions DESC
"""

top_agencies_type_of_idc = ps.sqldf(query, locals())
top_agencies_type_of_idc.to_excel('top_agencies_type_of_idc.xlsx', index=False)
top_agencies_type_of_idc

,awarding_agency_name,type_of_idc,number_of_transactions,total_obligation
0,Department of Health and Human Services,None,74,1.508402e+08
1,Department of Health and Human Services,INDEFINITE DELIVERY / INDEFINITE QUANTITY,2,0.000000e+00
2,Department of Justice,None,1440,2.231935e+09
3,Department of Justice,INDEFINITE DELIVERY / DEFINITE QUANTITY,1,0.000000e+00
4,Department of Veterans Affairs,None,723,4.516496e+08
5,Department of Veterans Affairs,INDEFINITE DELIVERY / REQUIREMENTS,5,0.000000e+00


In [15]:
query="""
SELECT
    awarding_agency_name,
    type_of_contract_pricing,
    COUNT(*) AS number_of_transactions,
    SUM(total_dollars_obligated) AS total_obligation
FROM primes
WHERE awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
GROUP BY awarding_agency_name, type_of_contract_pricing
ORDER BY awarding_agency_name, total_obligation DESC, number_of_transactions DESC
"""

top_agencies_type_of_contract_pricing = ps.sqldf(query, locals())
top_agencies_type_of_contract_pricing.to_excel('top_agencies_type_of_contract_pricing.xlsx', index=False)
top_agencies_type_of_contract_pricing

,awarding_agency_name,type_of_contract_pricing,number_of_transactions,total_obligation
0,Department of Health and Human Services,FIRM FIXED PRICE,72,1.308961e+08
1,Department of Health and Human Services,TIME AND MATERIALS,1,1.989015e+07
2,Department of Health and Human Services,LABOR HOURS,2,5.399637e+04
3,Department of Health and Human Services,ORDER DEPENDENT (IDV ALLOWS PRICING ARRANGEMEN...,1,0.000000e+00
4,Department of Justice,FIRM FIXED PRICE,1437,2.229372e+09
5,Department of Justice,LABOR HOURS,4,2.563471e+06
6,Department of Veterans Affairs,FIRM FIXED PRICE,728,4.516496e+08


In [16]:
query="""
SELECT
    awarding_agency_name,
    purchase_card_as_payment_method,
    COUNT(*) AS number_of_transactions,
    SUM(total_dollars_obligated) AS total_obligation
FROM primes
WHERE awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
GROUP BY awarding_agency_name, purchase_card_as_payment_method
ORDER BY awarding_agency_name, total_obligation DESC, number_of_transactions DESC
"""

top_agencies_purchase_card_as_payment_method = ps.sqldf(query, locals())
top_agencies_purchase_card_as_payment_method.to_excel('top_agencies_purchase_card_as_payment_method.xlsx', index=False)
top_agencies_purchase_card_as_payment_method

,awarding_agency_name,purchase_card_as_payment_method,number_of_transactions,total_obligation
0,Department of Health and Human Services,NO,68,1.508402e+08
1,Department of Health and Human Services,None,8,0.000000e+00
2,Department of Justice,NO,1440,2.231935e+09
3,Department of Justice,None,1,0.000000e+00
4,Department of Veterans Affairs,NO,720,4.510480e+08
5,Department of Veterans Affairs,YES,2,6.015500e+05
6,Department of Veterans Affairs,None,6,0.000000e+00


#### Research the Prime Contractors that have contracts requiring them to hire sub-contractors

In [17]:
query = """
WITH totals AS (
    SELECT
        awarding_agency_name,
        subcontracting_plan,
        COUNT(*) AS number_of_transactions,
        SUM(total_dollars_obligated) AS total_obligation
    FROM primes
    WHERE awarding_agency_name IN ('Department of Justice', 'Department of Veterans Affairs', 'Department of Health and Human Services')
    GROUP BY awarding_agency_name, subcontracting_plan
),
total_sum AS (
    SELECT SUM(total_obligation) AS grand_total FROM totals
)
SELECT
    t.awarding_agency_name,
    t.subcontracting_plan,
    t.number_of_transactions,
    t.total_obligation,
    ROUND((t.total_obligation * 100.0 / ts.grand_total), 2) AS percentage_of_total
FROM totals t
CROSS JOIN total_sum ts
ORDER BY t.awarding_agency_name DESC, t.subcontracting_plan DESC
"""
top_agencies_subcontracting_plan = ps.sqldf(query, locals())
top_agencies_subcontracting_plan.to_excel('top_agencies_subcontracting_plan.xlsx', index=False)
top_agencies_subcontracting_plan

,awarding_agency_name,subcontracting_plan,number_of_transactions,total_obligation,percentage_of_total
0,Department of Veterans Affairs,PLAN NOT REQUIRED,289,1.186587e+08,4.19
1,Department of Veterans Affairs,PLAN NOT INCLUDED - NO SUBCONTRACTING POSSIBIL...,425,3.041642e+08,10.73
2,Department of Veterans Affairs,INDIVIDUAL SUBCONTRACT PLAN,5,2.366674e+07,0.83
3,Department of Veterans Affairs,COMMERCIAL SUBCONTRACT PLAN,8,5.159940e+06,0.18
4,Department of Veterans Affairs,None,1,0.000000e+00,0.00
5,Department of Justice,PLAN NOT REQUIRED,180,1.940534e+08,6.85
6,Department of Justice,PLAN NOT INCLUDED - NO SUBCONTRACTING POSSIBIL...,6,2.343884e+05,0.01
7,Department of Justice,INDIVIDUAL SUBCONTRACT PLAN,1255,2.037647e+09,71.89
8,Department of Health and Human Services,PLAN REQUIRED - INCENTIVE INCLUDED,1,2.665856e+04,0.00
9,Department of Health and Human Services,PLAN NOT REQUIRED,34,9.988548e+07,3.52


In [18]:
for i in primes.columns:
    print(i)

contract_transaction_unique_key
contract_award_unique_key
award_id_piid
modification_number
transaction_number
parent_award_agency_id
parent_award_agency_name
parent_award_id_piid
parent_award_modification_number
federal_action_obligation
total_dollars_obligated
total_outlayed_amount_for_overall_award
base_and_exercised_options_value
current_total_value_of_award
base_and_all_options_value
potential_total_value_of_award
disaster_emergency_fund_codes_for_overall_award
outlayed_amount_from_COVID-19_supplementals_for_overall_award
obligated_amount_from_COVID-19_supplementals_for_overall_award
outlayed_amount_from_IIJA_supplemental_for_overall_award
obligated_amount_from_IIJA_supplemental_for_overall_award
action_date
action_date_fiscal_year
period_of_performance_start_date
period_of_performance_current_end_date
period_of_performance_potential_end_date
ordering_period_end_date
solicitation_date
awarding_agency_code
awarding_agency_name
awarding_sub_agency_code
awarding_sub_agency_name
award

In [21]:
set(primes['transaction_description'])

{'EMERGENCY HOUSING FOR SINGLE MALE AND FEMALE VETERANS UNDER THE HEALTHCARE FOR HOMELESS VETERANS (HCHV) PROGRAM, IN THE CATCHMENT AREA OF CHILLICOTHE VAMC.',
 'LACTATION CONSULTANT SERVICES',
 'RRC AND HC SERVICES IN MACON, GEORGIA.',
 'VALLHS 15-BED TRANSITIONAL HOUSING PROGRAM.',
 'EXERCISE OP 3 HCHV EMERGENCY TEMPORARY CONTRACTED HOUSING',
 'RRC AND HC SERVICES IN THE SALT LAKE CITY, UT AREA. OY3 & OY4',
 'RRC SERVICES IN KEARNEY, NEBRASKA. DE-OBLIGATE FY22 FUNDS',
 'TASK ORDER FOR RRC AND HC SERVICES FOR WACO, TX FROM 10/01/2022 TO 09/30/2023. THE PURPOSE OF THIS MODIFICATION IS TO PROVIDE ADDITIONAL FUNDING FOR OPTION YEAR THREE.',
 'HEALTHCARE FOR HOMELESS VETERANS - TRANSITIONAL HOUSING WACO/AUSTIN, TEXAS.',
 'RRC AND HC SERVICES FOR AUSTIN, TX',
 'COUNTRY CLIPS - BARBER SERVICES APRIL - SEPTEMBER FY23 $15,092.00',
 'ADD LINE ITEMS, PERIOD OF PERFORMANCE CHANGE, OPTION PERIOD EXERCISED, MOVE FY23 FUNDS.',
 'EMERGENCY VETERAN HOUSING',
 'HOMELESS VETERANS SAFE HAVEN SERVICES',


In [ ]:
keywords = ['Halfway Home', 'Foster', 'Safehome', 'Programming', 'Lived Experience']  # Replace with your desired keywords

# Create a case-insensitive search pattern
pattern = '|'.join(keywords)

# Filter the dataset based on keywords in the specified columns
filtered_df = primes[primes[['transaction_description']].apply(
    lambda col: col.str.contains(pattern, case=False, na=False)
).any(axis=1)]

filtered_df

,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,total_dollars_obligated,total_outlayed_amount_for_overall_award,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,disaster_emergency_fund_codes_for_overall_award,outlayed_amount_from_COVID-19_supplementals_for_overall_award,obligated_amount_from_COVID-19_supplementals_for_overall_award,outlayed_amount_from_IIJA_supplemental_for_overall_award,obligated_amount_from_IIJA_supplemental_for_overall_award,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,period_of_performance_potential_end_date,ordering_period_end_date,solicitation_date,awarding_agency_code,awarding_agency_name,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_agency_code,funding_agency_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,funding_office_name,treasury_accounts_funding_this_award,federal_accounts_funding_this_award,object_classes_funding_this_award,program_activities_funding_this_award,foreign_funding,foreign_funding_description,sam_exception,sam_exception_description,recipient_uei,recipient_duns,recipient_name,recipient_name_raw,recipient_doing_business_as_name,cage_code,recipient_parent_uei,recipient_parent_duns,recipient_parent_name,recipient_parent_name_raw,recipient_country_code,recipient_country_name,recipient_address_line_1,recipient_address_line_2,recipient_city_name,prime_award_transaction_recipient_county_fips_code,recipient_county_name,prime_award_transaction_recipient_state_fips_code,recipient_state_code,recipient_state_name,recipient_zip_4_code,prime_award_transaction_recipient_cd_original,prime_award_transaction_recipient_cd_current,recipient_phone_number,recipient_fax_number,primary_place_of_performance_country_code,primary_place_of_performance_country_name,primary_place_of_performance_city_name,prime_award_transaction_place_of_performance_county_fips_code,primary_place_of_performance_county_name,prime_award_transaction_place_of_performance_state_fips_code,primary_place_of_performance_state_code,primary_place_of_performance_state_name,primary_place_of_performance_zip_4,prime_award_transaction_place_of_performance_cd_original,prime_award_transaction_place_of_performance_cd_current,award_or_idv_flag,award_type_code,award_type,idv_type_code,idv_type,multiple_or_single_award_idv_code,multiple_or_single_award_idv,type_of_idc_code,type_of_idc,type_of_contract_pricing_code,type_of_contract_pricing,transaction_description,prime_award_base_transaction_description,action_type_code,action_type,solicitation_identifier,number_of_actions,inherently_governmental_functions,inherently_governmental_functions_description,product_or_service_code,product_or_service_code_description,contract_bundling_code,contract_bundling,dod_claimant_program_code,dod_claimant_program_description,naics_code,naics_description,recovered_materials_sustainability_code,recovered_materials_sustainability,domestic_or_foreign_entity_code,domestic_or_foreign_entity,dod_acquisition_program_code,dod_acquisition_program_description,information_technology_commercial_item_category_code,information_technology_commercial_item_category,epa_designated_product_code,epa_designated_product,country_of_product_or_service_origin_code,country_of_product_or_service_origin,place_of_manufacture_code,place_of_manufacture,subcontracting_plan_code,subcontracting_plan,extent_competed_code,extent_competed,solicitation_procedures_code,solicitation_procedures,type_of_set_aside_code,type_of_set_aside,evaluated_preference_code,evaluated_preference,research_code,research,fair_opportunity_limited_sources_code,fair_opportunity_limited_sources,other_than_full_and_open_competition_code,